In [1]:
from sentence_transformers import SentenceTransformer, util
# import mammoth # THIS IS FOR CONVERTING THE DOCX TO MD
from os import path, listdir
import re

In [2]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

In [3]:
%run load_docx.ipynb

In [36]:
file = 'path/to/story.docx'

In [37]:
file_name, file_extension = path.splitext(file)

In [38]:
if file_extension == '.docx':
    # doc = processDoc(file)
    md = processMD(file)
    # with open(file, "rb") as docx_file:
    #     md = mammoth.convert_to_markdown(docx_file)
        # len(doc[2:3][0])
print(md.messages)

[Message(type='warning', message='Unrecognised paragraph style: Title (Style ID: Title)')]


In [39]:
doc = md.value.split('\n\n')

In [40]:
def embed (string):
    return model.encode(string)

In [41]:
doc_embeddings = [(s,embed(s)) for s in doc]

In [42]:
def getType (md, w):
    principle = 'Purpose/ reason: Why is this important?'
    tactic = r'Tactic description: When do you apply this tactic\? How does it work\?'
    skill = r'Skills description: What does the ability help you do, achieve, or produce\?'
    tool = r'(Type of tool: If it is a tool, what type of tool is it\?|Short description: Describe in one sentence what the tool\/ method is, and what it does\.|Description of the tool: what is it\?)'

    res_principles = len(re.findall(principle, md))
    res_tactics = len(re.findall(tactic, md))
    res_skills = len(re.findall(skill, md))
    res_tools = len(re.findall(tool, md))

    # if res_principles == 0 and res_tactics == 0 and res_skills == 0 and res_tools == 0:
    print(w)
    print('pinciples', res_principles)
    print('tactics', res_tactics)
    print('skills', res_skills)
    print('tools', res_tools)
    print('\n')

    l = [('principles', res_principles), ('tactics', res_tactics), ('skills', res_skills), ('tools', res_tools)]
    return [t for (t,c) in l if c == max(res_principles, res_tactics, res_skills, res_tools)]

In [11]:
def filterMD (md):
    # REMOVE ALL THE CHECKLISTS
    md = re.sub(r'\#\#\#\# [\w\s\:\?\/\(\)\\;&]+\n', '', md)
    md = re.sub(r'\#+', '', md)
    md = re.sub(r'\n-[\s_]+(Sense|Explore|Develop|Test|Diffuse|Catalyze)', '', md)
    md = re.sub(r'\n-[\s_]+(Exploration|Solutions mapping|Experimentation)', '', md)
    md = re.sub(r'\n-[\s_]+(The advocate|The analyst|The communicator|The convener|The hustler|The project manager|The qualitative researcher|The visionary)', '', md)
    md = re.sub(r'\n-[\s_]+(Framework, Model|Guide, Manual|Platform, Portal|Playbook, Toolkit|Worksheet, Canvas|Process|Pilots|Proof of concept|Prototyping)', '', md)
    md = re.sub(r'\n-[\s_]+(Yes|No)', '', md)
    md = re.sub(r'[_\\]+', ' ', md)
    md = re.sub(r'\s+', ' ', md)
    return md.strip()

In [12]:
fromDB = False

In [23]:
if fromDB == True:
    %run get_keywords.ipynb
else:
    source = '/Users/myjyby/Documents/Projects/Codification fest/Transcripts/Elements/Docs/'
    seed_keywords = [(f, processMD(path.join(source, f)).value) for f in listdir(source) if f != '.DS_Store']

In [24]:
if fromDB == False:
    seed_keywords = [(w, filterMD(b), getType(b, w)[0]) for (w, b) in seed_keywords]

Recognize and promote agency.docx
pinciples 1
tactics 0
skills 0
tools 0


Have good references for your technical work.docx
pinciples 0
tactics 1
skills 0
tools 0


Play do not preach.docx
pinciples 0
tactics 1
skills 0
tools 0


Agility.docx
pinciples 1
tactics 0
skills 0
tools 0


Urbanist AI.docx
pinciples 0
tactics 0
skills 0
tools 1


Conducting interviews.docx
pinciples 0
tactics 0
skills 0
tools 1


Collective intelligence for the analysis of emerging signals.docx
pinciples 0
tactics 0
skills 2
tools 7


Literature review and desk research.docx
pinciples 0
tactics 0
skills 1
tools 1


The Experimenters Inventory.docx
pinciples 0
tactics 0
skills 0
tools 1


Storyboarding.docx
pinciples 0
tactics 0
skills 0
tools 0


Making insights actionable.docx
pinciples 0
tactics 0
skills 3
tools 0


A_B testing.docx
pinciples 0
tactics 0
skills 0
tools 2


Quick and continuous learning.docx
pinciples 1
tactics 0
skills 1
tools 0


Care about the people the same amount you care about the so

In [25]:
# print([(w,t) for (w,b,t) in seed_keywords])

In [26]:
for w,b,t in seed_keywords:
    if len(b) < 50:
        print(w, len(b))

seed_keywords = [(w,b,t) for (w,b,t) in seed_keywords if len(b) > 50]

Agility.docx 7
Expand through other voices.docx 29
Accessibility of knowledge.docx 26
Solving operational logistics.docx 30
Manage expectations.docx 19
Frugality.docx 11
Gaining leverage and traction for solutions.docx 45
Setting up the first block of the pyramid.docx 41
Diffusion of intelligence insight.docx 32
Sell De-risk.docx 15
Focus to creating a precedent.docx 29
Resilience.docx 10
Data sharing.docx 14
Pitch for impact.docx 18
Persuasion.docx 10
Digitise data collected in the field.docx 42
Always be proactive never be afraid of failure.docx 47
Collaboration.docx 44
Co-design.docx 42


In [27]:
seed_keywords[11]

('Quick and continuous learning.docx',
 'Quick and continuous learning Skills description: What does the ability help you do, achieve, or produce? To stay current and relevant - to be able to adapt - Continuous learning',
 'principles')

In [28]:
embedTitle = False

if embedTitle:
    keyword_embeddings = [(w,t,embed(w)) for (w,b,t) in seed_keywords]
else:
    keyword_embeddings = [(w,t,embed(b)) for (w,b,t) in seed_keywords]

In [29]:
def compare (e1, e2):
    return util.pytorch_cos_sim(e1, e2).tolist()[0][0]

In [43]:
topTags = []
useTypes = False

output_string = ''

for snippet in doc_embeddings:
    if len(snippet[0]) > 100 and not snippet[0].startswith('#') :
        distances = [(w,t,compare(e, snippet[1])) for (w,t,e) in keyword_embeddings]
        distances.sort(key = lambda tup: tup[2], reverse = True)
        types = list(set([t for (w,t,e) in keyword_embeddings]))
        
        if useTypes:
            for t in types:
                top_keyword = [w for (w, dt, e) in distances if dt == t][0]
                keyword, extension = path.splitext(top_keyword)
                output_string = output_string + "[[{}:{}]]".format(t.replace('capture card', '').strip().lower(), keyword)
                output_string = output_string + '\n'
        else:
            # print([(w, e) for (w, dt, e) in distances][0:3])
            top_keyword = [(w, dt) for (w, dt, e) in distances][:4]
            # output_string = output_string + "[[{}]]".format(top_keyword)
            for w, dt in top_keyword:
                keyword, extension = path.splitext(w)
                output_string = output_string + "[[{}:{}]]".format(dt, keyword)
                output_string = output_string + '\n'
            # output_string = output_string + '\n\n'

        output_string = output_string + snippet[0]
        output_string = output_string + '\n\n'
        
        topTags += [w for (w,t,e) in distances[:5]]
        output_string = output_string + '\n'
        # print(snippet[0], distances[:5])
    else:
        output_string = output_string + snippet[0]
        output_string = output_string + '\n\n'
print(output_string)

Explorer\-Togo\-Komi

# Code

- __Bold: __interviewer
- Regular: respondent

# Starts here

## Could you tell a bit about yourself, your name and your position?

[[tactics:Beyond borders]]
[[tactics:Hustle your gig]]
[[tactics:Getting in there]]
[[skills:Leveraging technologies]]
Thank you very much for this opportunity\. My name is Komi Aokou\. I work for UNDP Togo as Head of Exploration within that Accelerator Lab\.


## Well, hello, Komi, nice to meet you\.

Nice to meet you too\.

## Great\. So, could you tell me more about an R&D activity that you would like to share to me today?

[[skills:Innovation management]]
[[tactics:Systems thinking]]
[[tactics:Creating space for emerging themes]]
[[tactics:Hustle your gig]]
Of course, I would call that R&D activity U innovation\. So, here's university and there's innovation\. So, the combination of the two from is called what are called U innovation\. So, the work that we're doing around this, this challenge is how to bring innovation to t

In [32]:
def findDuplicates (l):
    seen = set()
    duplicates = []

    for x in l:
        if x in seen and x not in duplicates:
            duplicates.append(x)
        else:
            seen.add(x)
    return duplicates

In [33]:
findDuplicates(topTags)

['Recognize local expertise.docx',
 'Prioritize transparency in methods and openness in sharing data and results.docx',
 'Collective intelligence for the analysis of emerging signals.docx',
 'GIS systems.docx',
 'Design.docx',
 'Transparency and Honesty.docx']

In [34]:
output_name = "{}.md".format('_'.join([t for t in file_name.split('/') if t != '..' and t != '.']))
output_name

'Explorers_MEX.md'

In [35]:
output_file = open("../stories/pages/{}".format(output_name), 'w')
output_file.write(output_string)
output_file.close()